In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [3]:
# Load data
train_data = pd.read_csv('data_processed/train.csv')
val_data = pd.read_csv('data_processed/val.csv')

In [4]:
# Prepare data
X_train = train_data.drop(columns=['Result']).values
y_train = train_data['Result'].values
X_val = val_data.drop(columns=['Result']).values
y_val = val_data['Result'].values

In [5]:
train_data

,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,wins,draws,losses,oppos_wins,...,fouls,yellowcards,redcards,oppos_goals,oppos_shots,oppos_shotontarget,oppos_cornerkicks,oppos_fouls,oppos_yellowcards,oppos_redcards
0,1,31,0,3.042857,3.285714,2.341429,-0.4,0.0,0.4,0.000000,...,0.2,0.2,0.0,-0.333333,-11.666667,-9.666667,-5.333333,4.000000,0.333333,0.000000
1,3,28,0,2.240000,3.214286,3.285714,-0.2,-0.2,0.4,0.000000,...,2.0,1.0,0.0,-1.333333,-4.333333,-4.666667,0.000000,4.333333,0.333333,0.333333
2,13,34,0,1.932857,3.328571,4.092857,0.0,0.2,-0.2,0.333333,...,4.4,0.6,0.0,0.333333,-2.333333,-2.000000,1.000000,-4.666667,-1.000000,0.333333
3,20,29,2,1.267143,5.321429,11.785714,0.2,0.0,-0.2,0.333333,...,1.2,-0.4,0.0,0.333333,4.333333,3.000000,0.666667,-3.000000,-0.333333,-0.333333
4,36,35,0,2.078571,3.242857,3.664286,0.2,-0.4,0.2,0.333333,...,-3.2,0.4,-0.2,-1.000000,-14.000000,-9.666667,0.333333,4.666667,1.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,10,1,1,1.676667,4.141667,4.773333,-0.4,0.6,-0.2,0.666667,...,-0.8,-1.2,-0.4,0.666667,-1.666667,0.000000,-3.333333,3.666667,-1.333333,0.000000
2810,8,26,2,2.383333,3.125000,3.235000,0.0,0.0,0.0,0.000000,...,0.0,0.6,0.0,-2.000000,-3.333333,-4.000000,2.333333,2.333333,-0.666667,0.000000
2811,33,16,0,3.586667,3.651667,2.018333,0.2,0.2,-0.4,0.000000,...,-4.6,-0.8,0.0,-0.333333,4.333333,1.333333,0.666667,-1.333333,0.666667,0.333333
2812,20,36,2,1.546667,4.158333,6.156667,-0.2,-0.2,0.4,0.000000,...,1.4,1.2,0.0,-1.666667,-7.000000,-3.000000,-4.000000,3.666667,0.333333,0.000000


In [6]:
val_data

,HomeTeam,AwayTeam,Result,Hodds,Dodds,Aodds,wins,draws,losses,oppos_wins,...,fouls,yellowcards,redcards,oppos_goals,oppos_shots,oppos_shotontarget,oppos_cornerkicks,oppos_fouls,oppos_yellowcards,oppos_redcards
0,12,34,0,2.140000,3.445000,3.483333,0.0,0.0,0.0,0.666667,...,-1.2,-0.8,-0.2,1.333333,6.000000,3.666667,4.333333,4.333333,1.000000,0.000000
1,11,26,2,1.978333,3.383333,3.973333,0.2,0.0,-0.2,0.000000,...,1.6,0.0,0.0,0.000000,-7.000000,-2.000000,-5.333333,4.666667,2.000000,0.000000
2,7,36,1,2.650000,3.121667,2.835000,-0.4,-0.2,0.6,0.333333,...,2.0,0.2,0.0,0.000000,-2.666667,0.666667,-1.666667,4.333333,1.333333,0.000000
3,33,0,0,5.913333,4.023333,1.583333,-0.4,0.0,0.4,0.333333,...,-2.6,0.6,0.2,-2.333333,-7.666667,-4.333333,-3.000000,-1.000000,0.333333,0.333333
4,22,17,0,5.266667,3.940000,1.655000,-0.4,0.4,0.0,0.333333,...,0.4,-0.4,0.0,-2.000000,-1.000000,0.333333,-2.666667,6.000000,0.333333,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,27,31,1,3.711667,3.486667,2.038333,0.4,-0.2,-0.2,0.000000,...,5.0,1.2,-0.2,-2.333333,4.000000,-3.000000,0.000000,0.333333,1.000000,0.333333
340,32,34,0,3.291667,3.558333,2.173333,-0.4,-0.2,0.6,0.333333,...,-1.6,0.0,-0.2,-1.666667,-9.666667,-4.333333,-7.666667,-2.666667,0.333333,0.000000
341,17,18,2,7.793333,5.553333,1.356667,0.0,0.2,-0.2,0.333333,...,-1.4,-0.2,0.0,0.666667,-2.333333,1.333333,-2.000000,3.666667,0.666667,-0.333333
342,6,19,0,17.578333,7.785000,1.151667,0.2,0.0,-0.2,0.666667,...,-0.6,1.0,0.2,0.333333,9.333333,6.000000,3.000000,-0.333333,-0.666667,0.000000


# Logistic Regression

In [ ]:
# Cài đặt tham số Grid Search cho LR
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],  # Hệ số điều chỉnh
    'solver': ['lbfgs', 'newton-cg', 'saga'],  # Các thuật toán tối ưu
    'class_weight': [None, 'balanced']  # Cân bằng lớp
}

grid_lr = GridSearchCV(LogisticRegression(multi_class='multinomial', max_iter=5000), 
                       param_grid=param_grid_lr, 
                       scoring='accuracy', 
                       cv=5)  # 5-fold cross-validation

grid_lr.fit(X_train, y_train)
print("Best Parameters for Logistic Regression:", grid_lr.best_params_)
#Best Parameters for Logistic Regression: {'C': 0.01, 'class_weight': None, 'solver': 'lbfgs'}

In [ ]:
# Logistic Regression
best_lr = grid_lr.best_estimator_
y_pred_lr = best_lr.predict(X_val)
print("Accuracy for Logistic Regression:", accuracy_score(y_val, y_pred_lr))
print(classification_report(y_val, y_pred_lr))

Accuracy for Logistic Regression: 0.5319767441860465
              precision    recall  f1-score   support

           0       0.55      0.55      0.55       126
           1       0.33      0.01      0.02        78
           2       0.52      0.81      0.63       140

    accuracy                           0.53       344
   macro avg       0.47      0.46      0.40       344
weighted avg       0.49      0.53      0.47       344



# Random Forest

In [ ]:
# Cài đặt tham số Grid Search cho RF
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Số lượng cây
    'max_depth': [None, 10, 20, 30],  # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],  # Số mẫu tối thiểu để chia nhánh
    'class_weight': [None, 'balanced']  # Cân bằng lớp
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), 
                       param_grid=param_grid_rf, 
                       scoring='accuracy', 
                       cv=5)

grid_rf.fit(X_train, y_train)
print("Best Parameters for Random Forest:", grid_rf.best_params_)
#Best Parameters for Random Forest: {'class_weight': None, 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 300}

In [ ]:
# Random Forest
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_val)
print("Accuracy for Random Forest:", accuracy_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))

Accuracy for Random Forest: 0.7674418604651163
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       126
           1       1.00      0.51      0.68        78
           2       0.70      0.90      0.79       140

    accuracy                           0.77       344
   macro avg       0.83      0.73      0.75       344
weighted avg       0.80      0.77      0.76       344



# XGBoost

In [ ]:
# Cài đặt tham số Grid Search cho XGB
param_grid_xgb = {
    'n_estimators': [100, 200, 300],  # Số lượng cây
    'max_depth': [3, 5, 7],  # Độ sâu tối đa của cây
    'learning_rate': [0.01, 0.1, 0.2],  # Tốc độ học
    'subsample': [0.8, 1.0],  # Tỷ lệ mẫu ngẫu nhiên
    'colsample_bytree': [0.8, 1.0]  # Tỷ lệ cột được chọn cho mỗi cây
}

grid_xgb = GridSearchCV(xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                        param_grid=param_grid_xgb, 
                        scoring='accuracy', 
                        cv=5)

grid_xgb.fit(X_train, y_train)
print("Best Parameters for XGBoost:", grid_xgb.best_params_)


In [ ]:
# XGBoost
best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_val)
print("Accuracy for XGBoost:", accuracy_score(y_val, y_pred_xgb))
print(classification_report(y_val, y_pred_xgb))